In [ ]:
!pip install kaleido cohere openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
!pip install google-generativeai
!pip install -q neo4j-driver
# !pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.6/196.6 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import google.generativeai as palm
import base64
# import json
# import gradio as gr
from neo4j import GraphDatabase
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module=__name__)

In [ ]:
def get_answer(input):

  palm.configure(api_key="your api key")

  defaults = {
    'model': 'models/text-bison-001',
    'temperature': 0.7,
    'candidate_count': 1,
    'top_k': 40,
    'top_p': 0.95,
    'max_output_tokens': 1024,
    'stop_sequences': [],
    'safety_settings': [{"category":"HARM_CATEGORY_DEROGATORY","threshold":1},{"category":"HARM_CATEGORY_TOXICITY","threshold":1},{"category":"HARM_CATEGORY_VIOLENCE","threshold":2},{"category":"HARM_CATEGORY_SEXUAL","threshold":2},{"category":"HARM_CATEGORY_MEDICAL","threshold":2},{"category":"HARM_CATEGORY_DANGEROUS","threshold":2}],
  }

  prompt = f"""You are an expert in converting English queries to Neo4j Cypher Graph Code! The Graph has the following node labels - Types and Entity. Types Node label has the following property &#39;name&#39; and Entity Node label has the following properties - name, pronunciation, and word. The graph has only one relationship type is_a


  Relationship &#39;is_a&#39; starts from Entity Nodes to the Types Nodes and not the other way around.

  For Example,

  Example 1 - Give me the alphabet with the pronunciation &quot;k&quot;, the Cypher command would be like this -

  &#39;&#39;&#39;match (n:Entity)
  where n.pronounciation = &#39;k&#39;
  return n
  &#39;&#39;&#39;

  Example 2 -  Give me a word containing the alphabet &quot;ক&quot;, the Cypher command would be like this -
  &#39;&#39;&#39; match (n:Entity)
  where n.name = &#39;ক&#39;
  return n.word
  &#39;&#39;&#39;
  Example 3- Give me the pronunciation of the alphabet &#39;&#39;ক&quot;, the Cypher command would be like this -
  &#39;&#39;&#39;match (n:Entity)
  where n.pronounciation = &#39;k&#39;
  return n.name
  &#39;&#39;&#39;

  Dont include &#39;&#39;&#39; and \n in the outpu{input}t
  """

  response = palm.generate_text(**defaults,prompt=prompt)
  return (response.result)

In [ ]:
get_answer("Give me the pronunciation of the alphabet 'ক'")

"match (n:Entity)\nwhere n.name = 'ক'\nreturn n.pronounciation"

In [ ]:
driver = GraphDatabase.driver("bolt://3.236.161.127:7687",
                              auth=("neo4j","movements-traffic-cables"))

In [ ]:
import re

In [ ]:
def extract_query_and_return_key(input_query_result):
    slash_n_pattern = r'[ \n]+'
    ret_pattern = r'RETURN\s+(.*)'
    replacement = ' '

    cleaned_query = re.sub(slash_n_pattern, replacement, input_query_result)
    if cleaned_query:
        match = re.search(ret_pattern, cleaned_query)
        if match:
            extracted_string = match.group(1)
        else:
            extracted_string = ""
    return cleaned_query, extracted_string

In [ ]:
extract_query_and_return_key(get_answer("Give me the pronunciation of the alphabet 'ক'"))

("match (n:Entity) where n.name = 'ক' return n.pronounciation", '')

In [ ]:
def run_cypher_on_neo4j(inp_query, inp_key):
    out_list = []
    with driver.session() as session:
        result = session.run(inp_query)
        for record in result:
            if inp_key:
                out_list.append(record[inp_key])
            else:
                out_list.append(record)
    driver.close()
    if len(out_list) > 1:
        return format_names_with_ampersand(out_list)
    else:
        return out_list[0]


In [ ]:
query_1, string_1 = extract_query_and_return_key(get_answer("Give me the number pronounciation of 'ক'"))

In [ ]:
run_cypher_on_neo4j(query_1, string_1)

<Record n.pronounciation='k'>

In [ ]:
# def generate_and_exec_cypher(input_query):
#     gen_query, gen_key = extract_query_and_return_key(get_answer(input_query))
#     return run_cypher_on_neo4j(gen_query, gen_key)

In [ ]:
# def chatbot(input, history=[]):
#     output = str(generate_and_exec_cypher(input))
#     history.append((input, output))
#     return history, history

In [ ]:
def format_names_with_ampersand(names):
    if len(names) == 0:
        return ""
    elif len(names) == 1:
        return names[0]
    else:
        formatted_names = ", ".join(names[:-1]) + " & " + names[-1]
        return formatted_names